In [1]:
# Import libraries 

import json
import os
import random

import cv2 as cv
import keras.backend as K
import numpy as np
import scipy.io

from keras.models import model_from_json
from custom_layers.scale_layer import Scale

from PIL import Image
import glob

import requests
import urllib.request
import time
from bs4 import BeautifulSoup
from twilio.rest import Client

Using TensorFlow backend.


In [2]:
# Load model from json and h5 files

json_file = open('models/ResNet152.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json, {'Scale': Scale})
model.load_weights('models/ResNet152.hdf5')

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
# Set image size parameters
img_width, img_height = 224, 224

# Load labels
cars_meta = scipy.io.loadmat('labels/cars_meta')
class_names = cars_meta['class_names']  
class_names = np.transpose(class_names)

# Option 1: Pull from folder
test_path = 'images/samplesA/'
samples = [f for f in os.listdir(test_path) if os.path.isfile(os.path.join(test_path, f)) and f.endswith('.jpg')]
samples = [os.path.join(test_path, f) for f in samples]

# Option 2: Pull from live feed

# Define lists
num_samples = len(samples)
results = []

# Make prediction
for i, image_name in enumerate(samples):
    filename = image_name
    print('Start processing image: {}'.format(filename))
    bgr_img = cv.imread(filename)
    bgr_img = cv.resize(bgr_img, (img_width, img_height), cv.INTER_CUBIC)
    rgb_img = cv.cvtColor(bgr_img, cv.COLOR_BGR2RGB)
    rgb_img = np.expand_dims(rgb_img, 0)
    preds = model.predict(rgb_img)
    prob = np.max(preds)
    class_id = np.argmax(preds)
    text = ('Predict: {}, prob: {}'.format(class_names[class_id][0][0], prob))
    results.append({'label': class_names[class_id][0][0], 'prob': '{:.4}'.format(prob)})
    cv.imwrite('images/{}_out.png'.format(i), bgr_img)   

# Print and save results to file
print(results)

with open('results.json', 'w') as file:
    json.dump(results, file, indent=4)

K.clear_session()

Start processing image: images/samplesA/\frame13.jpg
Start processing image: images/samplesA/\frame7.jpg
Start processing image: images/samplesA/\frame6.jpg
Start processing image: images/samplesA/\frame12.jpg
Start processing image: images/samplesA/\frame10.jpg
Start processing image: images/samplesA/\frame38.jpg
Start processing image: images/samplesA/\frame4.jpg
Start processing image: images/samplesA/\frame5.jpg
Start processing image: images/samplesA/\frame39.jpg
Start processing image: images/samplesA/\frame11.jpg
Start processing image: images/samplesA/\frame29.jpg
Start processing image: images/samplesA/\frame15.jpg
Start processing image: images/samplesA/\frame1.jpg
Start processing image: images/samplesA/\frame0.jpg
Start processing image: images/samplesA/\frame14.jpg
Start processing image: images/samplesA/\frame28.jpg
Start processing image: images/samplesA/\frame16.jpg
Start processing image: images/samplesA/\frame2.jpg
Start processing image: images/samplesA/\frame3.jpg
S

In [4]:
for r in results: 
    print (r)

{'label': 'Chevrolet Silverado 1500 Classic Extended Cab 2007', 'prob': '0.5636'}
{'label': 'Dodge Ram Pickup 3500 Crew Cab 2010', 'prob': '0.2567'}
{'label': 'Dodge Ram Pickup 3500 Crew Cab 2010', 'prob': '0.3585'}
{'label': 'Dodge Ram Pickup 3500 Crew Cab 2010', 'prob': '0.4056'}
{'label': 'Chevrolet Silverado 1500 Extended Cab 2012', 'prob': '0.8927'}
{'label': 'GMC Terrain SUV 2012', 'prob': '0.2603'}
{'label': 'GMC Savana Van 2012', 'prob': '0.9687'}
{'label': 'Chevrolet Traverse SUV 2012', 'prob': '0.3028'}
{'label': 'Dodge Ram Pickup 3500 Crew Cab 2010', 'prob': '0.3384'}
{'label': 'Dodge Ram Pickup 3500 Crew Cab 2010', 'prob': '0.7577'}
{'label': 'Ford F-450 Super Duty Crew Cab 2012', 'prob': '0.4611'}
{'label': 'Audi 100 Wagon 1994', 'prob': '0.2191'}
{'label': 'GMC Savana Van 2012', 'prob': '0.838'}
{'label': 'GMC Savana Van 2012', 'prob': '0.9588'}
{'label': 'Dodge Sprinter Cargo Van 2009', 'prob': '0.2916'}
{'label': 'Ford F-450 Super Duty Crew Cab 2012', 'prob': '0.5779'}


In [5]:
# Collect list of stolen cars 

# Set up scraper
url = 'https://www.stolencar.com/Report/Search'
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Create list 
stolen_cars = [x.string.split() for x in soup.findAll('a', {"class": "report-view"}) if not x.has_attr('title')]

In [6]:
stolen_cars

[['2017', 'Hyundai', 'Tucson'],
 ['2003', 'Chevrolet', 'Traiblazer'],
 ['2018', 'Toyota', 'Highlander'],
 ['2019', 'Honda', 'Accord'],
 ['2019', 'Honda', 'Accord'],
 ['2006', 'Chevrolet', '1500', 'Pickup'],
 ['1992', 'Toyota', 'Pickup'],
 ['2018', 'Nissan', 'Rogue'],
 ['2000', 'Saturn', '4', 'DOOR', 'MODLE'],
 ['1998', 'Ford', 'Expedition'],
 ['1998', 'Ford', 'Expedition'],
 ['1999', 'Mercedes-Benz', 'SLK-Class'],
 ['2010', 'Chevrolet', 'Camaro', 'SS'],
 ['2010', 'Chevrolet', 'Camaro', 'SS'],
 ['2013', 'Ford', 'Taurus'],
 ['2015', 'Honda', 'Civic'],
 ['2013', 'Ford', 'Focus'],
 ['2019', 'Kia', 'Sorrento'],
 ['2006', 'Chevrolet', 'Impala'],
 ['2015', 'Chevrolet', 'Equinox'],
 ['2015', 'Ford', 'Escape'],
 ['2005', 'Acura', 'RSX'],
 ['1996', 'Honda', 'Accord', 'LX'],
 ['2008', 'Toyota', 'Camry'],
 ['2006', 'Pontiac', 'G6']]

In [7]:
# Setup the alert function
def send_sms(msg, to):
    
    sid = "ID TOKEN HERE"
    auth_token = "ID TOKEN HERE"
    twilio_number = "PHONE NUMBER HERE"
    client = Client(sid, auth_token) 
    message = client.messages.create(body=msg, from_=twilio_number, to=to)

In [8]:
# Create list of found cars
spotted_brands = []
for i, car in enumerate(results):
    brand = results[i]['label'].split(' ', 1)[0]
    spotted_brands.append(brand)

In [9]:
# Alert the victim
victim_no = "+19176646008"

for brand in spotted_brands:
    test = any(e[1] == brand for e in stolen_cars)
    if test:        
        send_sms("{} spotted!".format(brand), victim_no)